# FINN - End-to-End Flow
-----------------------------------------------------------------
This notebook gives an overview about the end to end flow of FINN. From loading an ONNX model from Brevitas, followed by the numerous transformations in FINN and up to the generation of a bitstream that can be used to load an FPGA. 

We'll use the following showSrc function to print the source code for function calls in the Jupyter notebook.

In [5]:
import inspect

def showSrc(what):
    print("".join(inspect.getsourcelines(what)[0]))

## Overview
The notebook is based on the following diagram. 

![](finn-design-flow-example.svg)

The diagram visualizes the end-to-end flow of FINN. The cylinder-like fields show the state of the network representation in the respective step. The rectangular fields represent the transformations that are applied to the network to achieve a certain result. The diagram is divided into 5 blocks, each of it includes several flow steps. The flow starts in top left corner with Brevitas export (pink block), followed by the preparation of the network (grey block) for the Vivado HLS and Vivado synthesis (yellow block). There is also a section for testing and verification in software (green block) and the hardware test on the PYNQ board (red block).
The diagram leads to the following outline for this Jupyter notebook.

## Outline
-------------
1. [Brevitas export](#brev_exp)
2. [Network preparation](#nw_prep)
    * Basic transformations
    * Streamlining
    * Conversion to HLS layers
    * Folding
3. [Vivado HLS and Vivado synthesis](#vivado)
    * HLS IP per layer
    * Creation of stitched design
    * PYNQ shell project
    * Synthesis, place and route
4. [Hardware Test](#hw_test)
5. [Simulation & Emulation flows for functional verification](#sim)
    * Simulation using Python
    * Simulation (npysim) using C++
    * Emulation (rtlsim) using PyVerilator

## 1. Brevitas export <a id='brev_exp'></a>
FINN expects an ONNX model as input. This can be a model trained with [Brevitas](https://github.com/Xilinx/brevitas). Brevitas is a Pytorch library for quantization-aware training and the FINN Docker image comes with several [example Brevitas networks](https://github.com/maltanar/brevitas_cnv_lfc). To show the FINN end-to-end flow, we'll use the LFC-w1a1 model as example network. The Brevitas export is only briefly described here, for details see Jupyter notebook [3-FINN-Brevitas-network-import](3-FINN-Brevitas-network-import.ipynb).

First a few things have to be imported. Then the model can be loaded with the pretrained weights.

In [9]:
import torch
import brevitas.onnx as bo
from models.LFC import LFC

lfc = LFC(weight_bit_width=1, act_bit_width=1, in_bit_width=1)
trained_lfc_checkpoint = ("/workspace/brevitas_cnv_lfc/pretrained_models/LFC_1W1A/checkpoints/best.tar")
checkpoint = torch.load(trained_lfc_checkpoint, map_location="cpu")
lfc.load_state_dict(checkpoint["state_dict"])
bo.export_finn_onnx(lfc, (1, 1, 28, 28), "lfc_w1_a1.onnx")

The model was now exported, loaded with the pretrained weights and saved under the name "lfc_w1_a1.onnx".
To visualize the exported model, Netron can be used. Netron is a visualizer for neural networks and allows interactive investigation of network properties. For example, you can click on the individual nodes and view the properties.

In [10]:
import netron
netron.start("lfc_w1_a1.onnx", port=8081, host="0.0.0.0")


Stopping http://0.0.0.0:8081
Serving 'lfc_w1_a1.onnx' at http://0.0.0.0:8081


In [11]:
%%html
<iframe src="http://0.0.0.0:8081/" style="position: relative; width: 100%;" height="400"></iframe>

Now that we have the model in .onnx format, we can work with it using FINN. For that FINN `ModelWrapper` is used. It is a wrapper around the ONNX model which provides several helper functions to make it easier to work with the model. For details see Jupyter notebook [2-FINN-ModelWrapper](2-FINN-ModelWrapper.ipynb).

In [12]:
from finn.core.modelwrapper import ModelWrapper
model = ModelWrapper("lfc_w1_a1.onnx")

Now the model is prepared and it can be processed in different ways. The principle of FINN are analysis and transformation passes, which can be applied to the model. An analysis pass extracts specific information about the model and returns it to the user in the form of a dictionary. For more details see [4-FINN-HowToAnalysisPass](4-FINN-HowToAnalysisPass.ipynb). A transformation pass changes the model and returns the changed model back to the FINN flow, for more information about transformation passes see notebook [5-FINN-HowToTransformationPass](5-FINN-HowToTransformationPass.ipynb).

Since the goal in this notebook is to process the model to such an extent that a bitstream can be generated from it, the focus is on the transformations that are necessary for this. In the next section these are discussed in more detail.

## 2. Network preparation <a id='nw_prep'></a>

* Basic transformations
* Streamlining
* Conversion to HLS layers
* Folding

### Basic transformations
This section deals with the basic transformations, which are applied to the model like a kind of clean up. They do not appear in the diagram above, but they are applied in many steps in the FINN flow to postprocess the model after a transformation and/or to prepare it for the next transformation.

The basis transformations are:
* GiveUniqueNodeNames
* GiveReadableTensorNames
* 
*
*

### Streamlining

### Conversion to HLS layers

### Folding

## 3. Vivado HLS and Vivado synthesis <a id='vivado'></a>
* HLS IP per layer
* Creation of stitched design
* PYNQ shell project
* Synthesis, place and route

### HLS IP per layer

### Creation of stitched design

### Conversion to HLS layers

### Synthesis, place and route

## 4. Hardware test <a id='hw_test'></a>

## 5. Simulation & Emulation flows for functional verification <a id='sim'></a>
* Simulation using Python
* Simulation (npysim) using C++
* Emulation (rtlsim) using PyVerilator

### Simulation using Python

### Simulation (npysim) using C++

### Emulation (rtlsim) using Pyverilator